In [1]:
#import needed package
import asyncio
import httpx
import pandas as pd
import secret
import refresh
from time import perf_counter

In [2]:
#authtokens and headers
id_token = refresh.refreshed()
headers = {"content-type":"application/json", "x-api-key": secret.api_key, "id-token": id_token}

In [3]:
#scrape all
vinny_id = 62869

In [4]:
#api calls
sem = asyncio.Semaphore(100)
#scrape all id

async def get_result(vinny_id:int):
    async with sem:
        async with httpx.AsyncClient() as client:
            r = await client.get(f'https://bps-world.vincere.io/api/v2/contact/{vinny_id}', headers=headers)
            if r.status_code == 200:
                results.append(r.json())
                return
            else:
                return

#trigger main function
async def main():
    tasks = []
    vinny_id = 33103
    #gen_id = df['vinny_id']
    while vinny_id <= 34013:
        tasks.append(get_result(vinny_id))
        vinny_id += 1
    await asyncio.gather(*tasks, return_exceptions=True)

#future container for finding errors and stuff
results = []

In [5]:
#trigger the whole function
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')

time taken: 22.917186599999695  seconds


In [6]:
#turn the data list into a pandas dataframe
df = pd.DataFrame.from_dict(results, orient='columns')

In [7]:
#from pandas dataframe to excel
df.to_excel('complete_contact_vincere_data_200323.xlsx', index=False)